## library

In [2]:
import os
import glob
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import numpy as np
import random
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint, uniform, loguniform

import xgboost as xgb
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, f1_score, roc_auc_score
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

### Config

In [3]:
start_date_str = '2015-01-01'
end_date_str = '2017-03-01'
def generate_first_of_month_dates(start_date_str, end_date_str):
    # Convert the date strings to datetime objects
    start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
    end_date = datetime.strptime(end_date_str, "%Y-%m-%d")
    
    # List to store the first of month dates
    first_of_month_dates = []

    # Start from the first of the month of the start_date
    current_date = datetime(start_date.year, start_date.month, 1)
    while current_date <= end_date:
        current_year = current_date.year
        current_month = current_date.month
        if current_month <10:
            current_date_str = f"{current_year}_0{current_month}_01"
        else:
            current_date_str = f"{current_year}_{current_month}_01"
        # Append the date in yyyy-mm-dd format
        first_of_month_dates.append(current_date_str)
        
        # Move to the first of the next month
        if current_date.month == 12:
            current_date = datetime(current_date.year + 1, 1, 1)
        else:
            current_date = datetime(current_date.year, current_date.month + 1, 1)
    

    return first_of_month_dates

dates_str = generate_first_of_month_dates(start_date_str, end_date_str)
print(dates_str)

['2015_01_01', '2015_02_01', '2015_03_01', '2015_04_01', '2015_05_01', '2015_06_01', '2015_07_01', '2015_08_01', '2015_09_01', '2015_10_01', '2015_11_01', '2015_12_01', '2016_01_01', '2016_02_01', '2016_03_01', '2016_04_01', '2016_05_01', '2016_06_01', '2016_07_01', '2016_08_01', '2016_09_01', '2016_10_01', '2016_11_01', '2016_12_01', '2017_01_01', '2017_02_01', '2017_03_01']


### Load Gold Tables

#### Labels

In [4]:
### Ingesting data from Google Cloud Storage

bucket_name = "cs611_mle"
label_path_in_bucket = "Gold Layer/labels.csv"
label_gcs_path = f"gs://{bucket_name}/{label_path_in_bucket}"

label_pdf = pd.read_csv(label_gcs_path)
label_pdf

,msno,membership_start_date,membership_expire_date,churn
0,++3Z+W8OPnpbHYfrKwqRKN1bF83XEbxjdYUolhGdHZg=,2015-01-10,2015-02-09,0
1,++3Z+W8OPnpbHYfrKwqRKN1bF83XEbxjdYUolhGdHZg=,2015-02-07,2015-03-09,0
2,++3Z+W8OPnpbHYfrKwqRKN1bF83XEbxjdYUolhGdHZg=,2015-03-10,2015-04-09,0
3,++3Z+W8OPnpbHYfrKwqRKN1bF83XEbxjdYUolhGdHZg=,2015-04-09,2015-05-09,0
4,++3Z+W8OPnpbHYfrKwqRKN1bF83XEbxjdYUolhGdHZg=,2015-06-04,2015-07-04,0
...,...,...,...,...
2821182,zzz1Dc3P9s53HAowRTrm3fNsWju5yeN4YBfNDq7Z99Q=,2016-10-25,2016-11-24,0
2821183,zzz1Dc3P9s53HAowRTrm3fNsWju5yeN4YBfNDq7Z99Q=,2016-11-24,2016-12-24,0
2821184,zzz1Dc3P9s53HAowRTrm3fNsWju5yeN4YBfNDq7Z99Q=,2016-12-25,2017-01-24,0
2821185,zzz1Dc3P9s53HAowRTrm3fNsWju5yeN4YBfNDq7Z99Q=,2017-01-25,2017-02-24,0


In [5]:
label_pdf = label_pdf.drop_duplicates(subset = ['msno', 'membership_start_date', 'membership_expire_date'], keep='first', inplace=False)
label_pdf

,msno,membership_start_date,membership_expire_date,churn
0,++3Z+W8OPnpbHYfrKwqRKN1bF83XEbxjdYUolhGdHZg=,2015-01-10,2015-02-09,0
1,++3Z+W8OPnpbHYfrKwqRKN1bF83XEbxjdYUolhGdHZg=,2015-02-07,2015-03-09,0
2,++3Z+W8OPnpbHYfrKwqRKN1bF83XEbxjdYUolhGdHZg=,2015-03-10,2015-04-09,0
3,++3Z+W8OPnpbHYfrKwqRKN1bF83XEbxjdYUolhGdHZg=,2015-04-09,2015-05-09,0
4,++3Z+W8OPnpbHYfrKwqRKN1bF83XEbxjdYUolhGdHZg=,2015-06-04,2015-07-04,0
...,...,...,...,...
2821182,zzz1Dc3P9s53HAowRTrm3fNsWju5yeN4YBfNDq7Z99Q=,2016-10-25,2016-11-24,0
2821183,zzz1Dc3P9s53HAowRTrm3fNsWju5yeN4YBfNDq7Z99Q=,2016-11-24,2016-12-24,0
2821184,zzz1Dc3P9s53HAowRTrm3fNsWju5yeN4YBfNDq7Z99Q=,2016-12-25,2017-01-24,0
2821185,zzz1Dc3P9s53HAowRTrm3fNsWju5yeN4YBfNDq7Z99Q=,2017-01-25,2017-02-24,0


In [6]:
label_pdf['membership_expire_date'] = pd.to_datetime(label_pdf['membership_expire_date'], format='%Y-%m-%d')
label_pdf['membership_start_date'] = pd.to_datetime(label_pdf['membership_start_date'], format='%Y-%m-%d')

/tmp/ipykernel_169184/170679981.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  label_pdf['membership_expire_date'] = pd.to_datetime(label_pdf['membership_expire_date'], format='%Y-%m-%d')
/tmp/ipykernel_169184/170679981.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  label_pdf['membership_start_date'] = pd.to_datetime(label_pdf['membership_start_date'], format='%Y-%m-%d')


In [7]:
label_pdf['plan_days'] = label_pdf['membership_expire_date'] -  label_pdf['membership_start_date']
label_pdf['plan_days'].unique()

/tmp/ipykernel_169184/915469325.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  label_pdf['plan_days'] = label_pdf['membership_expire_date'] -  label_pdf['membership_start_date']


<TimedeltaArray>
['30 days', '31 days']
Length: 2, dtype: timedelta64[ns]

In [8]:
# check label data
print("-------------LABEL INFORMATION-------------")
label_pdf.info()
print("-------------LABEL MISSING VALUE CHECK-------------")
# check missing values
label_pdf.isnull().sum()

-------------LABEL INFORMATION-------------
<class 'pandas.core.frame.DataFrame'>
Index: 2811276 entries, 0 to 2821186
Data columns (total 5 columns):
 #   Column                  Dtype          
---  ------                  -----          
 0   msno                    object         
 1   membership_start_date   datetime64[ns] 
 2   membership_expire_date  datetime64[ns] 
 3   churn                   int64          
 4   plan_days               timedelta64[ns]
dtypes: datetime64[ns](2), int64(1), object(1), timedelta64[ns](1)
memory usage: 128.7+ MB
-------------LABEL MISSING VALUE CHECK-------------


msno                      0
membership_start_date     0
membership_expire_date    0
churn                     0
plan_days                 0
dtype: int64

#### Features

In [ ]:
from google.cloud import storage
import os

bucket_name = "cs611_mle"
feature_path_in_bucket = "datamart/gold/feature_store"
feature_gcs_path = [f"gs://{bucket_name}/{feature_path_in_bucket}/gold_featurestore_{date_str}.parquet/" for date_str in dates_str]

df_dict = {}
for path in feature_gcs_path:
    path_index = feature_gcs_path.index(path)
    date_str = dates_str[path_index]
    df_dict[f'df_{date_str}'] = pd.read_parquet(path)

In [ ]:
feature_pdf = pd.concat(df_dict.values(), ignore_index=True)
feature_pdf

,msno,membership_start_date,membership_expire_date,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,is_cancel,...,songs_played_50_ratio,songs_played_75_ratio,songs_played_985_ratio,songs_completion_ratio,last_first_7_days_total_secs_ratio,city,gender,registered_via,registration_init_time,account_age
0,++pSqgOqSB8laOm+RTW6NLTqsMVQ0egh4Rs5+GOSJrQ=,2015-01-06,2015-02-05,41,30,149.0,119.0,1,2015-01-05,0,...,0.180851,0.074468,0.031915,0.489362,6.039049,5,male,7,2014-06-06,214
1,+/FpDPXcukKgEmdj/aCkhLlZPaPtdxycw1z11D2OukM=,2015-01-24,2015-02-23,41,30,149.0,119.0,1,2015-01-22,0,...,0.008726,0.006981,0.006981,0.944154,0.000000,1,na,7,2011-12-24,1127
2,+/ehcny76b1qm/wjQdzzp7sUps2DKTj/jM0P4qR/uzI=,2015-01-07,2015-02-07,40,31,149.0,149.0,1,2015-01-08,0,...,0.016556,0.026490,0.006623,0.844371,0.089451,11,na,9,2014-01-26,346
3,+03FlGdx1ny5jO9r7j09K1WGamqEGQvXxG5DfDk+5Sg=,2015-01-27,2015-02-27,40,31,149.0,149.0,1,2015-01-28,0,...,0.188830,0.074468,0.066489,0.449468,0.537696,13,na,9,2014-08-26,154
4,+04cPuRXH4W/GxWnS4qcA4vfohaRZmsCAstl6HBXdAM=,2015-01-08,2015-02-08,40,31,149.0,149.0,1,2015-01-08,0,...,0.018868,0.018868,0.000000,0.905660,4.409878,6,female,9,2008-09-05,2316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3083151,rIvVL1XCPoJEDlJIhNJ+1OVHcYd8eazlNQbQRvo6WpA=,2017-03-01,2017-03-31,33,30,149.0,149.0,1,2017-02-28,0,...,0.027124,0.014162,0.007681,0.671627,1.232709,13,male,3,2013-02-28,1462
3083152,BPJwzH06GC73bKYASJM6j7Jyc83OMT85Kgh+1GaJhoo=,2017-03-01,2017-03-31,34,30,149.0,149.0,1,2017-02-28,0,...,0.010152,0.000000,0.005076,0.974619,1.403180,17,male,9,2008-06-20,3176
3083153,iZ4udx03rOmrOgpWktnb/bNvVAVFkhQYGNBpyZujUtU=,2017-03-01,2017-03-31,41,30,99.0,99.0,1,2017-02-28,0,...,0.047198,0.038348,0.094395,0.551622,1.313007,22,female,7,2016-05-30,275
3083154,ylEDtz1yvNMyywS0dtv7N47FNGtKyaOZdaUakLbdC7g=,2017-03-01,2017-03-31,41,30,149.0,149.0,1,2017-02-28,0,...,0.078571,0.035714,0.021429,0.750000,22.289227,1,na,7,2015-03-29,703


In [ ]:
feature_pdf['plan_days'] = feature_pdf['membership_expire_date'] -  feature_pdf['membership_start_date']
feature_pdf['plan_days'].unique()

<TimedeltaArray>
[ '30 days',  '31 days', '195 days', '410 days', '360 days', '120 days',
 '180 days', '395 days', '100 days', '200 days', '450 days',  '70 days',
   '1 days',  '60 days', '365 days',  '90 days',   '0 days',  '21 days',
   '7 days',  '80 days',  '14 days', '230 days', '400 days',  '10 days',
  '15 days',  '35 days',  '99 days', '270 days',  '66 days', '240 days',
 '110 days',   '3 days',   '2 days',  '45 days']
Length: 34, dtype: timedelta64[ns]

In [ ]:
target_plan_days = ['31 days', '30 days']
feature_pdf = feature_pdf[feature_pdf['plan_days'].isin(target_plan_days)]
feature_pdf

/tmp/ipykernel_169118/4021510058.py:2: FutureWarning: The behavior of 'isin' with dtype=timedelta64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  feature_pdf = feature_pdf[feature_pdf['plan_days'].isin(target_plan_days)]


,msno,membership_start_date,membership_expire_date,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,is_cancel,...,songs_played_75_ratio,songs_played_985_ratio,songs_completion_ratio,last_first_7_days_total_secs_ratio,city,gender,registered_via,registration_init_time,account_age,plan_days
0,++pSqgOqSB8laOm+RTW6NLTqsMVQ0egh4Rs5+GOSJrQ=,2015-01-06,2015-02-05,41,30,149.0,119.0,1,2015-01-05,0,...,0.074468,0.031915,0.489362,6.039049,5,male,7,2014-06-06,214,30 days
1,+/FpDPXcukKgEmdj/aCkhLlZPaPtdxycw1z11D2OukM=,2015-01-24,2015-02-23,41,30,149.0,119.0,1,2015-01-22,0,...,0.006981,0.006981,0.944154,0.000000,1,na,7,2011-12-24,1127,30 days
2,+/ehcny76b1qm/wjQdzzp7sUps2DKTj/jM0P4qR/uzI=,2015-01-07,2015-02-07,40,31,149.0,149.0,1,2015-01-08,0,...,0.026490,0.006623,0.844371,0.089451,11,na,9,2014-01-26,346,31 days
3,+03FlGdx1ny5jO9r7j09K1WGamqEGQvXxG5DfDk+5Sg=,2015-01-27,2015-02-27,40,31,149.0,149.0,1,2015-01-28,0,...,0.074468,0.066489,0.449468,0.537696,13,na,9,2014-08-26,154,31 days
4,+04cPuRXH4W/GxWnS4qcA4vfohaRZmsCAstl6HBXdAM=,2015-01-08,2015-02-08,40,31,149.0,149.0,1,2015-01-08,0,...,0.018868,0.000000,0.905660,4.409878,6,female,9,2008-09-05,2316,31 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3083151,rIvVL1XCPoJEDlJIhNJ+1OVHcYd8eazlNQbQRvo6WpA=,2017-03-01,2017-03-31,33,30,149.0,149.0,1,2017-02-28,0,...,0.014162,0.007681,0.671627,1.232709,13,male,3,2013-02-28,1462,30 days
3083152,BPJwzH06GC73bKYASJM6j7Jyc83OMT85Kgh+1GaJhoo=,2017-03-01,2017-03-31,34,30,149.0,149.0,1,2017-02-28,0,...,0.000000,0.005076,0.974619,1.403180,17,male,9,2008-06-20,3176,30 days
3083153,iZ4udx03rOmrOgpWktnb/bNvVAVFkhQYGNBpyZujUtU=,2017-03-01,2017-03-31,41,30,99.0,99.0,1,2017-02-28,0,...,0.038348,0.094395,0.551622,1.313007,22,female,7,2016-05-30,275,30 days
3083154,ylEDtz1yvNMyywS0dtv7N47FNGtKyaOZdaUakLbdC7g=,2017-03-01,2017-03-31,41,30,149.0,149.0,1,2017-02-28,0,...,0.035714,0.021429,0.750000,22.289227,1,na,7,2015-03-29,703,30 days


In [ ]:
# check label data
print("-------------Feature INFORMATION-------------")
feature_pdf.info()
print("-------------Feature MISSING VALUE CHECK-------------")
# check missing values
feature_pdf.isnull().sum()

-------------Feature INFORMATION-------------
<class 'pandas.core.frame.DataFrame'>
Index: 2972482 entries, 0 to 3083155
Data columns (total 35 columns):
 #   Column                              Dtype          
---  ------                              -----          
 0   msno                                object         
 1   membership_start_date               datetime64[ns] 
 2   membership_expire_date              datetime64[ns] 
 3   payment_method_id                   object         
 4   payment_plan_days                   int32          
 5   plan_list_price                     float32        
 6   actual_amount_paid                  float32        
 7   is_auto_renew                       int32          
 8   transaction_date                    datetime64[ns] 
 9   is_cancel                           int32          
 10  discount_ratio                      float64        
 11  sum_completion_25                   int64          
 12  sum_completion_50                   int64  

msno                                  0
membership_start_date                 0
membership_expire_date                0
payment_method_id                     0
payment_plan_days                     0
plan_list_price                       0
actual_amount_paid                    0
is_auto_renew                         0
transaction_date                      0
is_cancel                             0
discount_ratio                        0
sum_completion_25                     0
sum_completion_50                     0
sum_completion_75                     0
sum_completion_985                    0
sum_completion_100                    0
sum_songs                             0
sum_total_secs                        0
avg_unique_song_played_ratio          0
total_secs_first_7_days               0
total_secs_last_7_days                0
avg_seconds_per_songs                 0
skip_ratio                            0
songs_played_25_ratio                 0
songs_played_50_ratio                 0


In [ ]:
feature_pdf = feature_pdf.drop_duplicates(subset = ['msno', 'membership_start_date', 'membership_expire_date'], keep='first', inplace=False)
feature_pdf

,msno,membership_start_date,membership_expire_date,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,is_cancel,...,songs_played_75_ratio,songs_played_985_ratio,songs_completion_ratio,last_first_7_days_total_secs_ratio,city,gender,registered_via,registration_init_time,account_age,plan_days
0,++pSqgOqSB8laOm+RTW6NLTqsMVQ0egh4Rs5+GOSJrQ=,2015-01-06,2015-02-05,41,30,149.0,119.0,1,2015-01-05,0,...,0.074468,0.031915,0.489362,6.039049,5,male,7,2014-06-06,214,30 days
1,+/FpDPXcukKgEmdj/aCkhLlZPaPtdxycw1z11D2OukM=,2015-01-24,2015-02-23,41,30,149.0,119.0,1,2015-01-22,0,...,0.006981,0.006981,0.944154,0.000000,1,na,7,2011-12-24,1127,30 days
2,+/ehcny76b1qm/wjQdzzp7sUps2DKTj/jM0P4qR/uzI=,2015-01-07,2015-02-07,40,31,149.0,149.0,1,2015-01-08,0,...,0.026490,0.006623,0.844371,0.089451,11,na,9,2014-01-26,346,31 days
3,+03FlGdx1ny5jO9r7j09K1WGamqEGQvXxG5DfDk+5Sg=,2015-01-27,2015-02-27,40,31,149.0,149.0,1,2015-01-28,0,...,0.074468,0.066489,0.449468,0.537696,13,na,9,2014-08-26,154,31 days
4,+04cPuRXH4W/GxWnS4qcA4vfohaRZmsCAstl6HBXdAM=,2015-01-08,2015-02-08,40,31,149.0,149.0,1,2015-01-08,0,...,0.018868,0.000000,0.905660,4.409878,6,female,9,2008-09-05,2316,31 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3083151,rIvVL1XCPoJEDlJIhNJ+1OVHcYd8eazlNQbQRvo6WpA=,2017-03-01,2017-03-31,33,30,149.0,149.0,1,2017-02-28,0,...,0.014162,0.007681,0.671627,1.232709,13,male,3,2013-02-28,1462,30 days
3083152,BPJwzH06GC73bKYASJM6j7Jyc83OMT85Kgh+1GaJhoo=,2017-03-01,2017-03-31,34,30,149.0,149.0,1,2017-02-28,0,...,0.000000,0.005076,0.974619,1.403180,17,male,9,2008-06-20,3176,30 days
3083153,iZ4udx03rOmrOgpWktnb/bNvVAVFkhQYGNBpyZujUtU=,2017-03-01,2017-03-31,41,30,99.0,99.0,1,2017-02-28,0,...,0.038348,0.094395,0.551622,1.313007,22,female,7,2016-05-30,275,30 days
3083154,ylEDtz1yvNMyywS0dtv7N47FNGtKyaOZdaUakLbdC7g=,2017-03-01,2017-03-31,41,30,149.0,149.0,1,2017-02-28,0,...,0.035714,0.021429,0.750000,22.289227,1,na,7,2015-03-29,703,30 days


#### Final Set

In [ ]:
final_pdf = pd.merge(label_pdf, feature_pdf, how = 'left', on = ['msno', 'membership_start_date', 'membership_expire_date'])
final_pdf = final_pdf.dropna()
final_pdf = final_pdf.reset_index()
final_pdf = final_pdf.drop(columns = ['index', 'plan_days_x', 'plan_days_y', 'transaction_date'])

In [ ]:
final_pdf['registered_via'] = final_pdf['registered_via'].astype('category')
final_pdf['churn'] = final_pdf['churn'].astype('category')
final_pdf['payment_method_id'] = final_pdf['payment_method_id'].astype('category')
final_pdf['is_auto_renew'] = final_pdf['is_auto_renew'].astype('category')
final_pdf['is_cancel'] = final_pdf['is_cancel'].astype('category')
final_pdf['city'] = final_pdf['city'].astype('category')
final_pdf['gender'] = final_pdf['gender'].astype('category')

In [ ]:
# check label data
print("-------------Final Set INFORMATION-------------")
final_pdf.info()
print("-------------Final Set VALUE CHECK-------------")
# check missing values
final_pdf.isnull().sum()

-------------Final Set INFORMATION-------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2624310 entries, 0 to 2624309
Data columns (total 34 columns):
 #   Column                              Dtype         
---  ------                              -----         
 0   msno                                object        
 1   membership_start_date               datetime64[ns]
 2   membership_expire_date              datetime64[ns]
 3   churn                               category      
 4   payment_method_id                   category      
 5   payment_plan_days                   float64       
 6   plan_list_price                     float32       
 7   actual_amount_paid                  float32       
 8   is_auto_renew                       category      
 9   is_cancel                           category      
 10  discount_ratio                      float64       
 11  sum_completion_25                   float64       
 12  sum_completion_50                   float64       

msno                                  0
membership_start_date                 0
membership_expire_date                0
churn                                 0
payment_method_id                     0
payment_plan_days                     0
plan_list_price                       0
actual_amount_paid                    0
is_auto_renew                         0
is_cancel                             0
discount_ratio                        0
sum_completion_25                     0
sum_completion_50                     0
sum_completion_75                     0
sum_completion_985                    0
sum_completion_100                    0
sum_songs                             0
sum_total_secs                        0
avg_unique_song_played_ratio          0
total_secs_first_7_days               0
total_secs_last_7_days                0
avg_seconds_per_songs                 0
skip_ratio                            0
songs_played_25_ratio                 0
songs_played_50_ratio                 0


### Prepare for Modelling

In [ ]:
OOT_pdf = final_pdf[(final_pdf['membership_start_date'] >= '2017-01-01')]
model_pdf = final_pdf[(final_pdf['membership_start_date'] < '2017-01-01')]

In [ ]:
y_model_ps = model_pdf['churn']
x_model_pdf = model_pdf.drop(columns = ['msno', 'membership_start_date', 'membership_expire_date', 'churn', 'registration_init_time'])

y_oot = OOT_pdf['churn']
x_oot = OOT_pdf.drop(columns = ['msno', 'membership_start_date', 'membership_expire_date', 'churn', 'registration_init_time'])


#### Train, Validation, Test Splitting

In [ ]:
# test
x_train, x_test,y_train, y_test = train_test_split(x_model_pdf, y_model_ps, test_size= 0.1, shuffle=True, random_state=611, stratify=y_model_ps)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size= 0.1, shuffle=True, random_state=611, stratify=y_train)

### EDA on train set

In [ ]:
print(x_train.info())

<class 'pandas.core.frame.DataFrame'>
Index: 1936335 entries, 453166 to 286520
Data columns (total 29 columns):
 #   Column                              Dtype   
---  ------                              -----   
 0   payment_method_id                   category
 1   payment_plan_days                   float64 
 2   plan_list_price                     float32 
 3   actual_amount_paid                  float32 
 4   is_auto_renew                       category
 5   is_cancel                           category
 6   discount_ratio                      float64 
 7   sum_completion_25                   float64 
 8   sum_completion_50                   float64 
 9   sum_completion_75                   float64 
 10  sum_completion_985                  float64 
 11  sum_completion_100                  float64 
 12  sum_songs                           float64 
 13  sum_total_secs                      float64 
 14  avg_unique_song_played_ratio        float64 
 15  total_secs_first_7_days          

In [ ]:
cate_cols = ['payment_method_id', 'is_auto_renew', 'is_cancel', 'city', 'gender', 'registered_via']
x_train_numeric = x_train.drop(columns = cate_cols)
x_val_numeric = x_val.drop(columns = cate_cols)
x_test_numeric = x_test.drop(columns = cate_cols)
x_oot_numeric = x_oot.drop(columns = cate_cols)

x_train_cate = x_train[cate_cols]
x_val_cate = x_train[cate_cols]
x_test_cate = x_train[cate_cols]
x_oot_cate = x_train[cate_cols]

print(x_train_numeric.shape[1])
print(x_val_numeric.shape[1])
print(x_test_numeric.shape[1])
print(x_oot_numeric.shape[1])



23
23
23
23


In [ ]:
# Standardisation for Numeric Columns
scaler = StandardScaler()

transformer_stdscaler = scaler.fit(x_train_numeric)

x_train_num_processed = pd.DataFrame(transformer_stdscaler.transform(x_train_numeric), columns=x_train_numeric.columns, index = x_train_numeric.index)
x_val_num_processed = pd.DataFrame(transformer_stdscaler.transform(x_val_numeric),columns=x_val_numeric.columns, index = x_val_numeric.index)
x_test_num_processed = pd.DataFrame(transformer_stdscaler.transform(x_test_numeric), columns=x_test_numeric.columns, index = x_test_numeric.index)
x_oot_num_processed = pd.DataFrame(transformer_stdscaler.transform(x_oot_numeric),columns=x_oot_numeric.columns, index = x_oot_numeric.index)

In [ ]:
# Categorical Data
# check payment_method_id
x_train_cate['payment_method_id'].value_counts(normalize=True).cumsum() * 100

payment_method_id
41     58.259134
40     76.131248
36     81.812186
39     87.470866
37     92.830579
38     95.716599
33     96.684871
34     97.410365
30     98.067277
27     98.559134
31     98.970116
29     99.348770
28     99.484025
24     99.618971
23     99.733362
16     99.822861
18     99.892942
21     99.934361
19     99.958530
25     99.982080
14     99.992460
10     99.996643
11     99.999948
7     100.000000
2     100.000000
Name: proportion, dtype: float64

In [ ]:
# deal with city
methods = ['41', '40', '36', '39', '37']

def method_mapping(col):
    if col in methods:
        return col
    else:
        return '99'
     

x_train_cate['payment_method_id'] = x_train_cate['payment_method_id'].apply(method_mapping)
x_val_cate['payment_method_id'] = x_val_cate['payment_method_id'].apply(method_mapping)
x_test_cate['payment_method_id'] = x_test_cate['payment_method_id'].apply(method_mapping)
x_oot_cate['payment_method_id'] = x_oot_cate['payment_method_id'].apply(method_mapping)

/tmp/ipykernel_169118/2366186560.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_train_cate['payment_method_id'] = x_train_cate['payment_method_id'].apply(method_mapping)
/tmp/ipykernel_169118/2366186560.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_val_cate['payment_method_id'] = x_val_cate['payment_method_id'].apply(method_mapping)
/tmp/ipykernel_169118/2366186560.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [ ]:
# Categorical Data
# check cities
x_train_cate['city'].value_counts(normalize=True).cumsum() * 100

city
1      43.197174
13     57.354177
5      67.508102
4      74.152613
15     79.637563
22     85.110738
6      88.278939
14     90.933232
12     92.387784
9      93.778711
18     94.777557
11     95.770257
10     96.709041
8      97.628200
17     98.283871
3      98.922242
21     99.514960
7      99.821932
16     99.911379
20     99.982751
19    100.000000
Name: proportion, dtype: float64

In [ ]:
# deal with city
cities = ['1', '13', '5', '4', '15', '22']

def city_mapping(col):
    if col in cities:
        return col
    else:
        return '99'
     

x_train_cate['city'] = x_train_cate['city'].apply(city_mapping)
x_val_cate['city'] = x_val_cate['city'].apply(city_mapping)
x_test_cate['city'] = x_test_cate['city'].apply(city_mapping)
x_oot_cate['city'] = x_oot_cate['city'].apply(city_mapping)

/tmp/ipykernel_169118/1406409206.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_train_cate['city'] = x_train_cate['city'].apply(city_mapping)
/tmp/ipykernel_169118/1406409206.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_val_cate['city'] = x_val_cate['city'].apply(city_mapping)
/tmp/ipykernel_169118/1406409206.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

In [ ]:
# Categorical Data
# check registered_via
x_train_cate['registered_via'].value_counts(normalize=True).cumsum() * 100

registered_via
7      60.948648
9      87.738227
3      98.877519
4      99.985798
13     99.999948
10    100.000000
Name: proportion, dtype: float64

In [30]:
# deal with registered_via
regist = ['7', '9', '3']

def regist_mapping(col):
    if col in regist:
        return col
    else:
        return '99'
     

x_train_cate['registered_via'] = x_train_cate['registered_via'].apply(regist_mapping)
x_val_cate['registered_via'] = x_val_cate['registered_via'].apply(regist_mapping)
x_test_cate['registered_via'] = x_test_cate['registered_via'].apply(regist_mapping)
x_oot_cate['registered_via'] = x_oot_cate['registered_via'].apply(regist_mapping)

/tmp/ipykernel_169118/3087510377.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_train_cate['registered_via'] = x_train_cate['registered_via'].apply(regist_mapping)
/tmp/ipykernel_169118/3087510377.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_val_cate['registered_via'] = x_val_cate['registered_via'].apply(regist_mapping)
/tmp/ipykernel_169118/3087510377.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

In [31]:
x_train_cate_processed = pd.get_dummies(x_train_cate, columns=['payment_method_id', 'city', 'gender', 'registered_via'], dtype=int)
x_val_cate_processed= pd.get_dummies(x_val_cate, columns=['payment_method_id', 'city', 'gender', 'registered_via'], dtype=int)
x_test_cate_processed = pd.get_dummies(x_test_cate, columns=['payment_method_id', 'city', 'gender', 'registered_via'], dtype=int)
x_oot_cate_processed = pd.get_dummies(x_oot_cate, columns=['payment_method_id', 'city', 'gender', 'registered_via'], dtype=int)

In [32]:
x_train = pd.concat([x_train_num_processed, x_train_cate_processed], axis=1)
x_val = pd.concat([x_val_num_processed, x_val_cate_processed], axis=1)
x_test = pd.concat([x_test_num_processed, x_test_cate_processed], axis=1)
x_oot = pd.concat([x_oot_num_processed, x_oot_cate_processed], axis=1)

In [ ]:
print(x_train.shape[0],x_train.shape[1])
print(x_val.shape[0],x_val.shape[1])
print(x_test.shape[0],x_test.shape[1])
print(x_oot.shape[0],x_oot.shape[1])

1936335 45
2151484 45
2175389 45
2170107 45


: 

### Modelling

#### Logistic Regression

In [ ]:
# Define the Random Forest classifier
lr_clf = LogisticRegression(random_state=88, solver='liblinear', max_iter=1000)

# Define the hyperparameter space to search
lr_param_dist = {
    'C': loguniform(0.01, 100),
    'penalty': ['l1', 'l2'], 
}

# Create a scorer based on AUC score
auc_scorer = make_scorer(roc_auc_score)

# Set up the random search with cross-validation
lr_random_search = RandomizedSearchCV(
    estimator=lr_clf,
    param_distributions=lr_param_dist,
    scoring=auc_scorer,
    n_iter=10,  # Number of iterations for random search
    cv=5,       # Number of folds in cross-validation
    verbose=1,
    random_state=42,
    n_jobs=-1   # Use all available cores
)

# Perform the random search
lr_random_search.fit(x_train, y_train)

# Output the best parameters and best score
print("Best parameters found: ", lr_random_search.best_params_)
print("Best AUC score: ", lr_random_search.best_score_)

# Evaluate the model on the train set
lr_best_model = lr_random_search.best_estimator_
y_pred_proba = lr_best_model.predict_proba(x_train)[:, 1]
lr_train_auc_score = roc_auc_score(y_train, y_pred_proba)
print("Train AUC score: ", lr_train_auc_score)

# Evaluate the model on the validation set
lr_best_model = lr_random_search.best_estimator_
y_pred_proba = lr_best_model.predict_proba(x_val)[:, 1]
lr_val_auc_score = roc_auc_score(y_val, y_pred_proba)
print("Validation AUC score: ", lr_val_auc_score)

# Evaluate the model on the test set
lr_best_model = lr_random_search.best_estimator_
y_pred_proba = lr_best_model.predict_proba(x_test)[:, 1]
lr_test_auc_score = roc_auc_score(y_test, y_pred_proba)
print("Test AUC score: ", lr_test_auc_score)

# Evaluate the model on the oot set
lr_best_model = lr_random_search.best_estimator_
y_pred_proba = lr_best_model.predict_proba(x_oot)[:, 1]
lr_oot_auc_score = roc_auc_score(y_oot, y_pred_proba)
print("OOT AUC score: ", lr_oot_auc_score)

print("TRAIN GINI score: ", round(2*lr_train_auc_score-1,3))
print("TRAIN GINI score: ", round(2*lr_val_auc_score-1,3))
print("Test GINI score: ", round(2*lr_test_auc_score-1,3))
print("OOT GINI score: ", round(2*lr_oot_auc_score-1,3))

In [ ]:
# Define the Random Forest classifier
rf_clf = RandomForestClassifier(random_state=88)

# Define the hyperparameter space to search
rf_param_dist = {
    'n_estimators': randint(50, 500),
    'max_depth': randint(3, 20),
    'max_features': ['sqrt', 'log2', 0.6, 0.8],
    'min_samples_leaf': randint(1, 10),
    'min_samples_split': randint(2, 20),
    'bootstrap': [True, False],
    'criterion': ['gini', 'entropy']
}

# Create a scorer based on AUC score
auc_scorer = make_scorer(roc_auc_score)

# Set up the random search with cross-validation
rf_random_search = RandomizedSearchCV(
    estimator=rf_clf,
    param_distributions=rf_param_dist,
    scoring=auc_scorer,
    n_iter=10,  # Number of iterations for random search
    cv=5,       # Number of folds in cross-validation
    verbose=1,
    random_state=42,
    n_jobs=-1   # Use all available cores
)

# Perform the random search
rf_random_search.fit(x_train, y_train)

# Output the best parameters and best score
print("Best parameters found: ", rf_random_search.best_params_)
print("Best AUC score: ", rf_random_search.best_score_)

# Evaluate the model on the train set
rf_best_model = rf_random_search.best_estimator_
y_pred_proba = rf_best_model.predict_proba(x_train)[:, 1]
rf_train_auc_score = roc_auc_score(y_train, y_pred_proba)
print("Train AUC score: ", rf_train_auc_score)

# Evaluate the model on the validation set
rf_best_model = rf_random_search.best_estimator_
y_pred_proba = rf_best_model.predict_proba(x_val)[:, 1]
rf_val_auc_score = roc_auc_score(y_val, y_pred_proba)
print("Test AUC score: ", rf_val_auc_score)

# Evaluate the model on the test set
rf_best_model = rf_random_search.best_estimator_
y_pred_proba = rf_best_model.predict_proba(x_test)[:, 1]
rf_test_auc_score = roc_auc_score(y_test, y_pred_proba)
print("Test AUC score: ", rf_test_auc_score)

# Evaluate the model on the oot set
rf_best_model = rf_random_search.best_estimator_
y_pred_proba = rf_best_model.predict_proba(x_oot)[:, 1]
rf_oot_auc_score = roc_auc_score(y_oot, y_pred_proba)
print("OOT AUC score: ", rf_oot_auc_score)

print("TRAIN GINI score: ", round(2*rf_train_auc_score-1,3))
print("Test GINI score: ", round(2*rf_test_auc_score-1,3))
print("OOT GINI score: ", round(2*rf_oot_auc_score-1,3))

Fitting 5 folds for each of 10 candidates, totalling 50 fits
